In [1]:
import numpy as np
import random
import datetime
import time
import os
from collections import defaultdict
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from tqdm import tqdm
import ast
import pandas as pd
import seaborn as sns

import argparse
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from ember_utils import *
from ember_model import *
from ember_pjr_utils import *

In [8]:

class Ember_MLP_Net_V2(nn.Module):
    def __init__(self):
        super(Ember_MLP_Net_V2, self).__init__()
        
        input_features = 2381
        self.fc1 = nn.Linear(input_features, 1024)
        self.fc1_bn = nn.BatchNorm1d(1024)
        self.act1 = nn.ReLU()
        self.fc1_drop = nn.Dropout(p=0.5)
        
        self.fc2 = nn.Linear(1024, 512)
        self.fc2_bn = nn.BatchNorm1d(512)
        self.act2 = nn.ReLU()
        self.fc2_drop = nn.Dropout(p=0.5)
        
        self.fc3 = nn.Linear(512, 256)
        self.fc3_bn = nn.BatchNorm1d(256)
        self.act3 = nn.ReLU()
        self.fc3_drop = nn.Dropout(p=0.5)        
        
        self.fc4 = nn.Linear(256, 128)
        self.fc4_bn = nn.BatchNorm1d(128)
        self.act4 = nn.ReLU()
        self.fc4_drop = nn.Dropout(p=0.5)  
        
        self.fc_last = nn.Linear(128, 1) 
        self.out = nn.Sigmoid()
        
        #self.activate = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        #print(x.shape)
        x = self.fc1(x)
        #x = self.fc1_bn(x)
        x = self.act1(x) 
        x = self.fc1_drop(x)

        x = self.fc2(x)
        #x = self.fc2_bn(x)
        x = self.act2(x) 
        x = self.fc2_drop(x)
        
        x = self.fc3(x)
        #x = self.fc3_bn(x)
        x = self.act3(x) 
        x = self.fc3_drop(x)
        
        x = self.fc4(x)
        #x = self.fc4_bn(x)
        x = self.act4(x)
        x = self.fc4_drop(x)
        
        x = self.fc_last(x)
        x = self.out(x)
        return x


def get_dataloader_weights(X, y, batch_size):
    
    X_ = torch.from_numpy(np.array(X)).type(torch.FloatTensor)
    y_ = torch.from_numpy(y).type(torch.FloatTensor)
    
    data_tensored = torch.utils.data.TensorDataset(X_,y_)    
    
    data_loader = torch.utils.data.DataLoader(data_tensored, batch_size = batch_size,
                                              num_workers=1, drop_last=False)
    return data_loader



def get_weights(model, layer, X_, Y_, batch_size, device):
    
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    
    if layer == 'fc2':
        model.fc2.register_forward_hook(get_activation(layer))
    elif layer == 'fc3':
        model.fc3.register_forward_hook(get_activation(layer))
    elif layer == 'fc4':
        model.fc4.register_forward_hook(get_activation(layer))
    elif layer == 'fc2_bn':
        model.fc2_bn.register_forward_hook(get_activation(layer))
    elif layer == 'fc3_bn':
        model.fc3_bn.register_forward_hook(get_activation(layer))
    elif layer == 'fc4_bn':
        model.fc4_bn.register_forward_hook(get_activation(layer))
    elif layer == 'act2':
        model.act2.register_forward_hook(get_activation(layer))
    elif layer == 'act3':
        model.act3.register_forward_hook(get_activation(layer))
    elif layer == 'act4':
        model.act4.register_forward_hook(get_activation(layer)) 
    
    dataloader = get_dataloader_weights(X_, Y_, batch_size)   
    
    model.eval()
    
    features = []

    with torch.no_grad():
        for x_batch, y_batch in tqdm(dataloader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            y_test_pred = model(x_batch)
            feats_batch = activation[layer].cpu().numpy()
            
            for f in feats_batch:
                features.append(f)
 
            
    assert len(features) == len(X_)      
    return np.array(features)



def GetFamilyDict(X_train, Y_train, Y_train_family,\
                  task_month, mal_cnt, global_family_dict):
    count = 0
    for x_ind, x_sample in enumerate(X_train):
        count += 1
        #print(x_ind, Y_train[x_ind])

        if Y_train[x_ind] == 0:
            global_family_dict["goodware"].append(x_sample)
        if Y_train[x_ind] == 1:
            mal_cnt += 1
            
            if Y_train_family[x_ind] == '':
                global_family_dict["others_family"].append(x_sample)
            else:
                global_family_dict[Y_train_family[x_ind]].append(x_sample)
    
    #print(f'Task {task_month} and #-of new samples stored {count}')
    
    return global_family_dict, mal_cnt


def AWS_Samples(v, v_weights, v_choose, get_anomalous=True, contamination=0.1):
    
    v_weights = np.array(v_weights) 
    data_X = v
    
    clf = IsolationForest(max_samples=len(v_weights), contamination=contamination)
    clf.fit(v_weights)
    y_pred = clf.predict(v_weights)
    anomalous_idx = np.where(y_pred == -1.0)
    similar_idx = np.where(y_pred == 1.0)

    assert len(anomalous_idx[0]) + len(similar_idx[0]) == len(y_pred)
    
    if get_anomalous:
        anomalous_samples_pool = list(data_X[anomalous_idx])
        similar_samples_pool = list(data_X[similar_idx])

        v_choose_split = int(np.ceil(v_choose/2))

        if len(anomalous_samples_pool) > v_choose_split:
            anomalous_samples = random.sample(anomalous_samples_pool, v_choose_split)

        else:
            anomalous_samples = anomalous_samples_pool

        if len(anomalous_samples) == v_choose_split:
            similar_samples = random.sample(similar_samples_pool, v_choose_split)
        elif len(anomalous_samples) < v_choose_split:
            v_choose_split += v_choose_split - len(anomalous_samples)
            if len(similar_samples_pool) > v_choose_split:
                similar_samples = random.sample(similar_samples_pool, v_choose_split)
            else:
                similar_samples = similar_samples_pool
        if len(anomalous_samples) > 0 and len(similar_samples) > 0: 
            anomalous_samples, similar_samples = np.array(anomalous_samples), np.array(similar_samples)
            #print(f'anomalous_samples {anomalous_samples.shape} similar_samples {similar_samples.shape}')
            replay_samples = np.concatenate((anomalous_samples, similar_samples))
        else:
            if len(anomalous_samples) <= 0:
                replay_samples = similar_samples
            if len(similar_samples) <= 0:
                replay_samples = anomalous_samples
    else:
        similar_samples_pool = list(data_X[similar_idx])
        if len(similar_samples_pool) > v_choose:
            similar_samples = random.sample(similar_samples_pool, v_choose)
        else:
            similar_samples = similar_samples_pool
            
        replay_samples = np.array(similar_samples)
        
    return replay_samples


def AWS(GFamilyDict, memory_budget, model, layer, batch_size, device,\
        goodware_ifs=False, min_samples=1, fs_option='ratio'):
    #fs_option = 'uniform'
    #memory_budget = 1000
    goodware_budget = malware_budget = int(np.ceil(memory_budget/2))
    
    num_families = len(GFamilyDict.keys()) - 1 
    pre_malSamples = []
    #cnt = 0
    #fam_cnt = 0
    
    for k, v in GFamilyDict.items():
        
        if k != 'goodware':
            if fs_option != 'gifs':
                #fam_cnt += 1
                v = np.array(v)
                #print(f'{k} - {len(v)}')
                #cnt += len(v)

                if fs_option == 'ratio':
                    v_choose = int(np.ceil((len(v) / malware_count) * malware_budget))
                
                if fs_option == 'uniform':
                    v_choose = int(np.ceil(malware_budget / num_families))
                
                if fs_option == 'mix':
                    v_choose = int(np.ceil((len(v) / malware_count) * malware_budget))
                    if v_choose < min_samples:
                        #print(f'v_choose {v_choose} min_samples {min_samples}')
                        v_choose = min_samples
                    #else: print(f'v_choose {v_choose} **')                

                if len(v) <= v_choose:
                    for i in v:
                        pre_malSamples.append(i)
                else:
                    v_Y = np.ones(len(v))
                    v_weights = get_weights(model, layer, v, v_Y, batch_size, device)
                    v = AWS_Samples(v, v_weights, v_choose, get_anomalous=True, contamination=0.1)
                    
                    for i in v:
                        pre_malSamples.append(i)
            else:
                for i in v:
                    pre_malSamples.append(i)
    
    if fs_option == 'gifs':
        if malware_budget < len(pre_malSamples):
            pre_malSamples = random.sample(list(pre_malSamples), malware_budget)
    
    
    all_Goodware = GFamilyDict['goodware']
    if goodware_ifs:
        #print(f'I am here NOW.')
        pre_GoodSamples = []
            
        if len(all_Goodware) > goodware_budget:
            v = all_Goodware
            v_Y = np.ones(len(v))
            v_weights = get_weights(model, layer, v, v_Y, batch_size, device)
            v_choose = goodware_budget
            v = AWS_Samples(v, v_weights, v_choose, get_anomalous=True, contamination=0.1)
            for i in v:
                pre_GoodSamples.append(i)
        else:
            for i in v:
                pre_GoodSamples.append(i)
    else:
        if goodware_budget > len(all_Goodware):
            pre_GoodSamples = all_Goodware
        else:
            pre_GoodSamples = random.sample(list(all_Goodware), goodware_budget)
    
    
    samples_to_replay = np.concatenate((np.array(pre_GoodSamples), np.array(pre_malSamples)))
    labels_to_replay = np.concatenate((np.zeros(len(pre_GoodSamples)), np.ones(len(pre_malSamples))))
    
    X_replay, Y_replay = shuffle(samples_to_replay, labels_to_replay)
    
    return X_replay, Y_replay


def MixSampleCount(GBudget, MinBudget, GFamilyDict):
    
    print(f'budget unallocated {GBudget} MinBudget {MinBudget}')
    
    tmpBudget = GBudget
    
    GfamStat = {}
    
    for fam, S in GFamilyDict.items():
        if fam != 'goodware':
            GfamStat[fam] = len(S)
    
    assert len(GfamStat.keys()) == len(GFamilyDict.keys()) - 1
    
    GfamChoose = {}
    GfamTemp = {}
    
    allocated = 0
    for fam, numSample in GfamStat.items():
        if numSample > MinBudget:
            GfamChoose[fam] = MinBudget
            GfamTemp[fam] = numSample - MinBudget
            GBudget -= MinBudget
            allocated += MinBudget
        else:
            GfamChoose[fam] = numSample
            GfamTemp[fam] = 0
            GBudget -= numSample
            allocated += numSample
    
    print(f'GBudget {GBudget} allocated {allocated}')
    
    if allocated > tmpBudget:
        print(f'reduce minimum samples, budget is lower than required allocation')
    
    UnallocatedSamples = int(sum(GfamTemp.values()))

    #print(f'allocated {allocated} unallocated {GBudget} Sample remainin {UnallocatedSamples}')
    for fam, numSample in GfamTemp.items():
        if numSample != 0:
            #print(f'here ')
            allocate = int(np.round((numSample/UnallocatedSamples) * GBudget))
            #print(f'GBudget {GBudget} {np.round(numSample/UnallocatedSamples)} allocate {allocate}')
            GfamChoose[fam] += allocate
            
    #print(f'_________________')        
            
    return GfamChoose




all_task_months = ['2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
                   '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12']

# data_dir = '../../month_based_processing_with_family_labels/'

patience = 5
replay_type = 'aws'


data_dir = '../../../month_based_processing_with_family_labels/' #args.data_dir
num_exps = 1 #args.num_exps
contamination = 0.1 #args.contamination #0.1 #[0.2, 0.3, 0.4, 0.5]

replay_type = ifs_option = 'ratio'
num_epoch = 500
batch_size = 6000
memory_budget = 400000
min_samples = 0
layer = 'act4'
batch_norm = 'no'


exp_seeds = [random.randint(1, 99999) for i in range(num_exps)]

use_cuda = True
print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
use_cuda = use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
torch.manual_seed(42)

model = Ember_MLP_Net()

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

model = model.to(device)
print(f'Model has {count_parameters(model)/1000000}m parameters')    
criterion = nn.BCELoss()    

GFamilyDict = defaultdict(list)
malware_count = 0

standardization = StandardScaler()
standard_scaler = None

for task_month in range(len(all_task_months[:2])):

    print(f'\n{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}...')
    task_start = time.time()
    
    #task_month = task_month
    current_task = all_task_months[task_month]
    task_months = all_task_months[:task_month+1]
    #print(f'Current Task {current_task} w/ {num_samples_per_malware_family} samples to Replay per Malware family.')


    model_save_dir = '../AWS_SavedModel' + '/AWSModel_' + str(memory_budget) + '/' + str(current_task) + '/'
    create_parent_folder(model_save_dir)

    opt_save_path = '../AWS_SavedModel' + '/AWSOpt_' + str(memory_budget) + '/' + str(current_task) + '/'
    create_parent_folder(opt_save_path)

    results_save_dir =  './AWS_SavedResults_' +'/AWS_' + str(memory_budget) + '/' 
    create_parent_folder(results_save_dir)


    X_train, Y_train, Y_train_family = get_family_labeled_month_data(data_dir, current_task)
    X_test, Y_test, Y_test_family = get_family_labeled_task_test_data(data_dir, task_months, mlp_net=True)


    if current_task == all_task_months[0]:
        GFamilyDict, malware_count = GetFamilyDict(X_train, Y_train, Y_train_family,\
                                               current_task, malware_count, GFamilyDict)
        num_Y_replay = 0
        print(f'\n\ntask {task_month} num_Y_replay {num_Y_replay}\n\n')
    else:
        model_new = Ember_MLP_Net_V2()
        model_new = model_new.to(device)
        #load the best model for this task
        #best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
        print(f'loading best model {best_model_path}')
        model_new.load_state_dict(torch.load(best_model_path))
        
        X_replay, Y_replay = AWS(GFamilyDict, memory_budget, model_new,\
                                          layer, batch_size, device,\
                                          goodware_ifs=True, min_samples=min_samples,
                                          fs_option=ifs_option)
        
        num_Y_replay = len(Y_replay)
        print(f'\n\ntask {task_month} num_Y_replay {num_Y_replay}\n\n')
        GFamilyDict, malware_count = GetFamilyDict(X_train, Y_train, Y_train_family,\
                                       current_task, malware_count, GFamilyDict)
    print()
    print(f'X_train {X_train.shape} Y_train {Y_train.shape}')
    print()
    
    X_train, Y_train = shuffle(X_train, Y_train)

    print()
    print(f'X_train {X_train.shape} Y_train {Y_train.shape}')
    print()

    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Standardizing ...')
    standard_scaler = standardization.partial_fit(X_train)

    X_train = standard_scaler.transform(X_train)
    X_test = standard_scaler.transform(X_test)

    X_train, Y_train = np.array(X_train, np.float32), np.array(Y_train, np.int32)
    X_test, Y_test = np.array(X_test, np.float32), np.array(Y_test, np.int32)


    print(f'{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")} Training ...')
    task_training_time, epoch_ran, training_loss, validation_loss  =\
                            training_early_stopping(model, model_save_dir, opt_save_path,\
                            X_train, Y_train, X_test, Y_test, patience,\
                            batch_size, device, optimizer, num_epoch,\
                             criterion, replay_type, current_task, 42, earlystopping=True)



    model = Ember_MLP_Net()
    model = model.to(device)
    #load the best model for this task
    best_model_path = model_save_dir + os.listdir(model_save_dir)[0]
    print(f'loading best model {best_model_path}')
    model.load_state_dict(torch.load(best_model_path))


    optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.000001)
    best_optimizer = opt_save_path + os.listdir(opt_save_path)[0]
    print(f'loading best optimizer {best_optimizer}')
    optimizer.load_state_dict(torch.load(best_optimizer))

    acc, precision, recall, f1score = testing_aucscore(model, X_test, Y_test, batch_size, device)
    
    print(f'test acc {acc}')
    

Torch 1.2.0+cu92 CUDA 9.2.148
Model has 3.132161m parameters

2023-08-19 22:35:56...
X_train (55722, 2381) Y_train (55722,) Y_tr_family (55722,)
X_test (6192, 2381) Y_test (6192,) Y_te_family (6192,)


task 0 num_Y_replay 0



X_train (55722, 2381) Y_train (55722,)


X_train (55722, 2381) Y_train (55722,)

2023-08-19 22:35:58 Standardizing ...


  0%|          | 0/9 [00:00<?, ?it/s]

2023-08-19 22:36:03 Training ...
Epoch 1 of 500


  0%|          | 0/9 [00:00<?, ?it/s]

Train Loss: 0.6980, Train Acc: 0.5265
Val Loss: 0.6871, Val Acc: 0.5762
Validation loss decreased (inf --> 0.687130).  Saving model ...
../AWS_SavedModel/AWSModel_400000/2018-01/best_model_epoch_1.pt
../AWS_SavedModel/AWSOpt_400000/2018-01/best_optimizer_epoch_1.pt
Epoch 2 of 500


...

  0%|          | 0/8 [00:00<?, ?it/s]

Train Loss: 0.2418, Train Acc: 0.9056
Val Loss: 0.1904, Val Acc: 0.9225
Validation loss decreased (0.195784 --> 0.190405).  Saving model ...
../AWS_SavedModel/AWSModel_400000/2018-02/best_model_epoch_2.pt
../AWS_SavedModel/AWSOpt_400000/2018-02/best_optimizer_epoch_2.pt
Epoch 3 of 500


100%|██████████| 2/2 [00:00<00:00,  7.98it/s]

Train Loss: 0.2225, Train Acc: 0.9110
Val Loss: 0.1818, Val Acc: 0.9284
Validation loss decreased (0.190405 --> 0.181840).  Saving model ...


KeyboardInterrupt: 

In [22]:


X_replay, Y_replay = AWS(GFamilyDict, memory_budget, model_new,\
                                          layer, batch_size, device,\
                                          goodware_ifs=True, min_samples=min_samples,
                                          fs_option=ifs_option)

print(f'Y_replay {Y_replay.shape}')

pre_GoodSamples (47058, 2381) pre_malSamples (57387, 2381)
Y_replay (104445,)


In [23]:


X_replay, Y_replay = AWS(GFamilyDict, memory_budget, model_new,\
                                          layer, batch_size, device,\
                                          goodware_ifs=True, min_samples=min_samples,
                                          fs_option='uniform')

print(f'Y_replay {Y_replay.shape}')

pre_GoodSamples (47058, 2381) pre_malSamples (57387, 2381)
Y_replay (104445,)


In [29]:
def AWS_Samples(v, v_weights, v_choose, get_anomalous=True, contamination=0.1):
    
    v_weights = np.array(v_weights) 
    data_X = v
    
    clf = IsolationForest(max_samples=len(v_weights), contamination=contamination)
    clf.fit(v_weights)
    y_pred = clf.predict(v_weights)
    anomalous_idx = np.where(y_pred == -1.0)
    similar_idx = np.where(y_pred == 1.0)

    assert len(anomalous_idx[0]) + len(similar_idx[0]) == len(y_pred)
    
    if get_anomalous:
        anomalous_samples_pool = list(data_X[anomalous_idx])
        similar_samples_pool = list(data_X[similar_idx])

        v_choose_split = int(np.ceil(v_choose/2))

        if len(anomalous_samples_pool) > v_choose_split:
            anomalous_samples = random.sample(anomalous_samples_pool, v_choose_split)

        else:
            anomalous_samples = anomalous_samples_pool

        if len(anomalous_samples) == v_choose_split:
            similar_samples = random.sample(similar_samples_pool, v_choose_split)
        elif len(anomalous_samples) < v_choose_split:
            v_choose_split += v_choose_split - len(anomalous_samples)
            if len(similar_samples_pool) > v_choose_split:
                similar_samples = random.sample(similar_samples_pool, v_choose_split)
            else:
                similar_samples = similar_samples_pool
        if len(anomalous_samples) > 0 and len(similar_samples) > 0: 
            anomalous_samples, similar_samples = np.array(anomalous_samples), np.array(similar_samples)
            #print(f'anomalous_samples {anomalous_samples.shape} similar_samples {similar_samples.shape}')
            replay_samples = np.concatenate((anomalous_samples, similar_samples))
        else:
            if len(anomalous_samples) <= 0:
                replay_samples = similar_samples
            if len(similar_samples) <= 0:
                replay_samples = anomalous_samples
    else:
        similar_samples_pool = list(data_X[similar_idx])
        if len(similar_samples_pool) > v_choose:
            similar_samples = random.sample(similar_samples_pool, v_choose)
        else:
            similar_samples = similar_samples_pool
            
        replay_samples = np.array(similar_samples)
        
    return replay_samples


def AWS(GFamilyDict, memory_budget, model, layer, batch_size, device,\
        goodware_ifs=False, min_samples=1, fs_option='ratio'):
    #fs_option = 'uniform'
    #memory_budget = 1000
    goodware_budget = malware_budget = int(np.ceil(memory_budget/2))
    
    num_families = len(GFamilyDict.keys()) - 1 
    pre_malSamples = []
    #cnt = 0
    #fam_cnt = 0
    
    for k, v in GFamilyDict.items():
        if k != 'goodware':
            #if malware_count > malware_budget:
            if fs_option != 'gifs':
                #fam_cnt += 1
                v = np.array(v)
                #print(f'{k} - {len(v)}')
                #cnt += len(v)

                if fs_option == 'ratio':
                    v_choose = int(np.ceil((len(v) / malware_count) * malware_budget))

                if fs_option == 'uniform':
                    v_choose = int(np.ceil(malware_budget / num_families))

                if fs_option == 'mix':
                    v_choose = int(np.ceil((len(v) / malware_count) * malware_budget))
                    if v_choose < min_samples:
                        #print(f'v_choose {v_choose} min_samples {min_samples}')
                        v_choose = min_samples
                    #else: print(f'v_choose {v_choose} **')                

                if len(v) <= v_choose:
                    print(f'len(v) {len(v)} <= v_choose {v_choose}')
                    for i in v:
                        pre_malSamples.append(i)
                else:
                    print(f'len(v) {len(v)} > v_choose {v_choose}')
                    v_Y = np.ones(len(v))
                    v_weights = get_weights(model, layer, v, v_Y, batch_size, device)
                    v = AWS_Samples(v, v_weights, v_choose, get_anomalous=True, contamination=0.1)

                    for i in v:
                        pre_malSamples.append(i)
            else:
                for i in v:
                    pre_malSamples.append(i)
#             else:
#                 for i in v:
#                     pre_malSamples.append(i)    

    if fs_option == 'gifs':
        if malware_budget < len(pre_malSamples):
            pre_malSamples = random.sample(list(pre_malSamples), malware_budget)
    
    
    all_Goodware = GFamilyDict['goodware']
    if goodware_ifs:
        #print(f'I am here NOW.')
        
        v = all_Goodware
        pre_GoodSamples = []
        if len(all_Goodware) > goodware_budget:
            v_Y = np.ones(len(v))
            v_weights = get_weights(model, layer, v, v_Y, batch_size, device)
            v_choose = goodware_budget
            v = AWS_Samples(v, v_weights, v_choose, get_anomalous=True, contamination=0.1)
            for i in v:
                pre_GoodSamples.append(i)
        else:
            for i in v:
                pre_GoodSamples.append(i)
    else:
        if goodware_budget > len(all_Goodware):
            pre_GoodSamples = all_Goodware
        else:
            pre_GoodSamples = random.sample(list(all_Goodware), goodware_budget)
    
    print(f'pre_GoodSamples {np.array(pre_GoodSamples).shape} pre_malSamples {np.array(pre_malSamples).shape}')
    samples_to_replay = np.concatenate((np.array(pre_GoodSamples), np.array(pre_malSamples)))
    labels_to_replay = np.concatenate((np.zeros(len(pre_GoodSamples)), np.ones(len(pre_malSamples))))
    
    X_replay, Y_replay = shuffle(samples_to_replay, labels_to_replay)
    
    return X_replay, Y_replay



memory_budget = 100000
X_replay, Y_replay = AWS(GFamilyDict, memory_budget, model_new,\
                                          layer, batch_size, device,\
                                          goodware_ifs=True, min_samples=1,
                                          fs_option='uniform')

print(f'Y_replay {Y_replay.shape}')

  0%|          | 0/1 [00:00<?, ?it/s]

len(v) 305 > v_choose 39


  0%|          | 0/1 [00:00<?, ?it/s]

len(v) 871 > v_choose 39


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 5 <= v_choose 39
len(v) 58 > v_choose 39


100%|██████████| 1/1 [00:00<00:00,  3.70it/s]


len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 19 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 4 <= v_choose 39
len(v) 12 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 8 <= v_choose 39
len(v) 4 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 21 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 9 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 1 <= v_choose 39
len(v) 2 <= v_choose 39
len(v) 3 <= v_choose 39
len(v) 1 <= v

pre_GoodSamples (47058, 2381) pre_malSamples (10675, 2381)
Y_replay (57733,)


57388